In [ ]:
pip install pycoingecko
!pip install plotly
!pip install mplfinance

In [1]:
#In this notebook, you will learn to convert an audio file of an English speaker 
#to text using a Speech to Text API. 
#Then you will translate the English version to a Spanish version using a Language Translator API. 
#Note: You must obtain the API keys and enpoints to complete the lab.


#you will need the following library 
!pip install ibm_watson wget

In [ ]:
#First we import SpeechToTextV1 from ibm_watson.

from ibm_watson import SpeechToTextV1 
import json
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
#The service endpoint is based on the location of the service instance, 
#we store the information in the variable URL. To find out which URL to use, 
#view the service credentials and paste the url here.


url_s2t = "https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/83ff186c-cf75-4f29-b918-b41dfaf70b0b"

In [ ]:
#You require an API key, and you can obtain the key on the Dashboard .
iam_apikey_s2t = "7EPV6GfNK5S2_J9juGrMZ4OQIC9O40PLL0qpPJgs7Wt2"

In [ ]:
#You create a Speech To Text Adapter object, the parameters are the endpoint and API key.

authenticator = IAMAuthenticator(iam_apikey_s2t)
s2t = SpeechToTextV1(authenticator=authenticator)
s2t.set_service_url(url_s2t)
s2t

In [ ]:
#Lets download the audio file that we will use to convert into text. 
#Can also manually downlaod and put into same directory as this program

!wget -O PolynomialRegressionandPipelines.mp3  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/data/PolynomialRegressionandPipelines.mp3

In [ ]:
#We have the path of the wav file we would like to convert to text
#If not in same directory as this program please use full file path

filename='PolynomialRegressionandPipelines.mp3'

In [ ]:
#We create the file object wav with the wav file using with/open ; 
#we set the mode to "rb", which is 'read in binary' mode 
#We use the method 'recognize' on the object s2t to return 'response' an object saving the Service response. 
#The parameter audio is the file object, wav
#the parameter content_type is the format of the file, audio/mp3


with open(filename, mode="rb")  as wav:
    response = s2t.recognize(audio=wav, content_type='audio/mp3')

In [ ]:
#outputs python dictionary containing the results 
response.result

In [ ]:
from pandas import json_normalize
#Normalize semi-structured JSON data into a flat table. does not touch underlying data
json_normalize(response.result['results'],"alternatives")

In [ ]:
#object type is unchanged
response.result

In [ ]:
#save a particular string in thedictionary.
#the 10 is the row index in which the string is located
#we look under the results key and find a list with the entries (length 11 in this case)
#we look at one of the values on the list (example index 10) and find a dictionary with 2 keys final and alternatives
#alternatives is a single entry list, so we look at its 0th entry, which is a dictionary with 2 keys transcript and confidence
#finally we lookup the value under the key transcipt: a string 
recognized_text=response.result['results'][10]["alternatives"][0]["transcript"]
type(recognized_text)


In [ ]:
recognized_text

In [ ]:
#Language Translator
#First we import LanguageTranslatorV3 from ibm_watson

from ibm_watson import LanguageTranslatorV3

In [ ]:
#The service endpoint is based on the location of the service instance, 
#we store the information in the variable URL. 
#To find out which URL to use, view the service credentials.

url_lt='https://api.us-south.language-translator.watson.cloud.ibm.com/instances/4c6101bd-c1c3-45bc-8c20-9a98d4488c07'

In [ ]:
#You require an API key, and you can obtain the key on the Dashboard.
apikey_lt='TrFC8NDBKLyLJ28GR-vzhwRrvxsLNkJA5n5Yk-9fWxMv'

In [ ]:
#API requests require a version parameter that takes a date in the format 
#version=YYYY-MM-DD. This lab describes the current version of Language Translator, 
#2018-05-01
version_lt='2018-05-01'

In [ ]:
#we create a Language Translator object, language_translator:
authenticator = IAMAuthenticator(apikey_lt)
language_translator = LanguageTranslatorV3(version=version_lt,authenticator=authenticator)
language_translator.set_service_url(url_lt)
language_translator

In [ ]:
#We can list the languages that the service can identify. 
#The method Returns the language code. For example English (en) to Spanis (es) 
#and name of each language.

json_normalize(language_translator.list_identifiable_languages().get_result(), "languages")

In [ ]:
#We can use the method translate this will translate the text. 
#The parameter text is the object name. Model_id are the languages we are translating from and to
#In this case, we set it to 'en-es' or English to Spanish. 
#We get a 'Detailed Response' object, and name it 'translation_response'

translation_response = language_translator.translate(text=recognized_text, model_id='en-es')
type(translation_response)

In [ ]:
#We use the get result method on the object -translation response- to get a dictionary. 
#The dictionary is saved to the variable translation

translation=translation_response.get_result()

translation

In [ ]:
type(translation)

In [ ]:
#We can obtain the actual translation as a string as follows:
#under translations key
#indexed entry on the list
#under the translation key is the string
spanish_translation =translation['translations'][0]['translation']
spanish_translation 

In [ ]:
#We can translate back to English
translation_new = language_translator.translate(text=spanish_translation ,model_id='es-en').get_result()

In [ ]:
#We can obtain the actual translation as a string as follows:
translation_eng=translation_new['translations'][0]['translation']
translation_eng

In [ ]:
#QUIZ #Translate to French.

French_translation=language_translator.translate(text=translation_eng , model_id='en-fr').get_result()

French_translation['translations'][0]['translation']